In [1]:
!pip install tensorflow keras sentence-transformers keras-tuner

  Using cached tensorflow-2.18.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (4.0 kB)
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached grpcio-1.68.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.9 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached numpy-2.0.2-cp311-cp311-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached ml_dtypes-0.4.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached huggingface_hub-0.26.5-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached numpy-1.26.4

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from kerastuner.tuners import Hyperband
# from tensorflow.keras.tuner import Hyperband  # Use Keras Tuner from TensorFlow
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
import torch



/var/folders/m9/4j5hjgcd5qngvl020z555w240000gn/T/ipykernel_69602/3529243669.py:12: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import Hyperband
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
df = pd.read_csv('huggingface_with_perplexity.csv')


In [4]:
# Check the total number of NaNs in each column
nan_summary = df.isna().sum()
print("\nCount of NaN values per column:")
print(nan_summary[nan_summary > 0])  # Show only columns with NaN values


Count of NaN values per column:
Series([], dtype: int64)


In [5]:
df = df[~df['perplexity'].isna()]


In [6]:
# Identify rows where 'generated' == 'generated'
rows_to_drop = df[df['generated'] == 'generated'].index

# Drop these rows
dataset = df.drop(rows_to_drop)

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# Extract the "text" column for similarity checking
texts = dataset["text"].tolist()

# Compute TF-IDF vectors for the text data
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

# Compute cosine similarity matrix
cosine_sim_matrix = cosine_similarity(tfidf_matrix)

# Remove duplicate-like rows based on a 90% similarity threshold
to_remove = set()
for i in range(len(texts)):
    for j in range(i + 1, len(texts)):
        if cosine_sim_matrix[i, j] > 0.9:  # 90% similarity threshold
            to_remove.add(j)

print(len(to_remove))
df_distinct = dataset.drop(list(to_remove)).reset_index(drop=True)
df_distinct = dataset.drop(list(to_remove)).reset_index(drop=True)
print(df_distinct.shape)

10380
(4620, 16)


In [8]:
df_distinct.shape

(4620, 16)

In [9]:
print("Is GPU available?", torch.cuda.is_available())

Is GPU available? False


In [11]:
model = SentenceTransformer('all-MiniLM-L6-v2')
text_embeddings = model.encode(df_distinct['text'].tolist(), show_progress_bar=True)


Batches: 100%|██████████| 145/145 [00:41<00:00,  3.48it/s]


In [69]:
print(text_embeddings)

[[-0.01686272 -0.01975334  0.00672336 ... -0.03179123  0.05161779
   0.04418319]
 [ 0.03201037  0.02006243  0.07152901 ...  0.03716383 -0.05485476
   0.06056327]
 [-0.01988794  0.09259275  0.08762163 ...  0.01249938 -0.05002529
   0.02056968]
 ...
 [ 0.04894435 -0.04333448  0.02027768 ...  0.07240842 -0.07655276
  -0.06850141]
 [ 0.00574559 -0.04813547  0.08095636 ...  0.03290918 -0.00781281
  -0.03662317]
 [ 0.06045514 -0.04425557 -0.0092147  ...  0.02052861  0.08309416
   0.08355667]]


In [12]:
embeddings_df = pd.DataFrame(text_embeddings)

In [103]:
print(embeddings_df.shape)

(15000, 384)


In [13]:
X = embeddings_df
y = df_distinct['generated']
y.astype(int)  # Target column
X = np.array(X, dtype=np.float32)  # Convert X to float32
y = np.array(y, dtype=np.float32)

In [14]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [15]:
assert not any(np.array_equal(X_train[i], X_test[j]) for i in range(len(X_train)) for j in range(len(X_test))), "Data Leakage Detected!"


In [16]:
# Find duplicates between X_train and X_test
duplicates = [np.array_equal(X_train[i], X_test[j]) for i in range(len(X_train)) for j in range(len(X_test))]

# Extract overlapping samples (optional for debugging)
overlapping_indices = [
    (i, j) for i in range(len(X_train)) for j in range(len(X_test)) if np.array_equal(X_train[i], X_test[j])
]

print(f"Number of overlapping samples: {len(overlapping_indices)}")


Number of overlapping samples: 0


In [17]:
print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Training set size: 2772
Validation set size: 924
Test set size: 924


In [48]:
def build_model(hp):
    model = Sequential([
        Input(shape=(X.shape[1],)),  # Input shape based on combined feature vector
        Dense(hp.Int('units_1', 32, 128, step=32), activation='relu', kernel_regularizer=l2(1e-4)),
        Dropout(hp.Float('dropout_1', 0.2, 0.6, step=0.1)),
        Dense(hp.Int('units_2', 16, 64, step=16), activation='relu', kernel_regularizer=l2(1e-4)),
        Dropout(hp.Float('dropout_2', 0.2, 0.6, step=0.1)),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-4, 1e-5, 1e-6])),
        loss='binary_crossentropy',
        metrics=['accuracy',Precision(), Recall(), AUC()]
    )
    return model


In [49]:
import os
import shutil

# Clear any
if os.path.exists('mlp_embed_tuning'):
    shutil.rmtree('mlp_embed_tuning')


tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=2,
    directory='mlp_embed_tuning',
    project_name='mlp_tune_embeddings'
)

In [50]:
tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32, verbose=2)



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
64                |64                |units_1
0.4               |0.4               |dropout_1
32                |32                |units_2
0.3               |0.3               |dropout_2
1e-06             |1e-06             |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round



KeyboardInterrupt: 

In [22]:
best_hps = tuner.get_best_hyperparameters(num_trials=10)[0]
print(f"Best hyperparameters: {best_hps.values}")

Best hyperparameters: {'units_1': 48, 'dropout_1': 0.30000000000000004, 'units_2': 24, 'dropout_2': 0.2, 'learning_rate': 0.0001, 'tuner/epochs': 20, 'tuner/initial_epoch': 10, 'tuner/bracket': 4, 'tuner/round': 4, 'tuner/trial_id': '0035'}


In [23]:
best_model = tuner.hypermodel.build(best_hps)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = best_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32, callbacks=[early_stopping])

Epoch 1/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4954 - auc_3: 0.4852 - loss: 0.7058 - precision_3: 0.4691 - recall_3: 0.1738 - val_accuracy: 0.5541 - val_auc_3: 0.6845 - val_loss: 0.6996 - val_precision_3: 0.8022 - val_recall_3: 0.1563
Epoch 2/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5512 - auc_3: 0.6362 - loss: 0.6977 - precision_3: 0.6453 - recall_3: 0.2209 - val_accuracy: 0.5920 - val_auc_3: 0.8559 - val_loss: 0.6872 - val_precision_3: 0.9327 - val_recall_3: 0.2077
Epoch 3/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6098 - auc_3: 0.7827 - loss: 0.6829 - precision_3: 0.8640 - recall_3: 0.2421 - val_accuracy: 0.6483 - val_auc_3: 0.9207 - val_loss: 0.6660 - val_precision_3: 0.9671 - val_recall_3: 0.3148
Epoch 4/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6542 - auc_3: 0.8611 - loss: 0.6606 - precision_3: 0.8995 - recall_3: 0.3342 - val_accuracy: 0.7110 - val_auc_3: 0.9553 - val_loss: 0.6329 - val_precision_3: 0.9762 - val_recall_3:

In [24]:
from collections import Counter

print("Training Set Distribution:", Counter(y_train))
print("Validation Set Distribution:", Counter(y_val))
print("Test Set Distribution:", Counter(y_test))

Training Set Distribution: Counter({0.0: 1408, 1.0: 1364})
Validation Set Distribution: Counter({1.0: 467, 0.0: 457})
Test Set Distribution: Counter({1.0: 479, 0.0: 445})


In [147]:
loss, accuracy,precision, recall, accuracy  = best_model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"Test AUC: {auc}")

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9825 - auc_7: 0.9989 - loss: 0.0693 - precision_7: 0.9869 - recall_7: 0.9785 
Test Loss: 0.07109230011701584
Test Accuracy: 0.9986770153045654
Test Precision: 0.9894291758537292
Test Recall: 0.9770354628562927
Test AUC: 0.8252246975898743


In [27]:
df_htest = pd.read_csv('Dataset_with_new_features.csv')

In [29]:
# Identify rows where 'generated' == 'generated'
rows_to_drop = df_htest[df_htest['generated'] == 'generated'].index

# Drop these rows
dataset_h = df_htest.drop(rows_to_drop)

In [30]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# Extract the "text" column for similarity checking
texts = dataset_h["text"].tolist()

# Compute TF-IDF vectors for the text data
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

# Compute cosine similarity matrix
cosine_sim_matrix = cosine_similarity(tfidf_matrix)

# Remove duplicate-like rows based on a 90% similarity threshold
to_remove = set()
for i in range(len(texts)):
    for j in range(i + 1, len(texts)):
        if cosine_sim_matrix[i, j] > 0.9:  # 90% similarity threshold
            to_remove.add(j)

print(len(to_remove))
df_h_distinct = dataset_h.drop(list(to_remove)).reset_index(drop=True)
df_h_distinct = dataset_h.drop(list(to_remove)).reset_index(drop=True)
print(df_h_distinct.shape)

184
(9426, 29)


In [31]:
text_embeddings_h = model.encode(df_h_distinct['text'].tolist(), show_progress_bar=True)


Batches: 100%|██████████| 295/295 [01:25<00:00,  3.47it/s]


In [43]:
X_h = text_embeddings_h
y_h = df_h_distinct['generated']
y_h.astype(int)  # Target column
X_h = np.array(X_h, dtype=np.float32)  # Convert X to float32
y_h = np.array(y_h, dtype=np.float32)

In [44]:
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report, precision_recall_curve
y_pred = (best_model.predict(X_h) > 0.5).astype(int)  # Predicted labels
y_prob = best_model.predict(X_h).ravel()  # Predicted probabilities
fpr, tpr, thresholds = roc_curve(y_h, y_prob)
print(fpr)
roc_auc = auc(fpr, tpr)
balanced_accuracy = 0.5 * (tpr.mean() + (1 - fpr).mean())

295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 356us/step
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 292us/step
[0.         0.         0.         ... 0.99981854 0.99981854 1.        ]


In [46]:
print(f"ROC AUC: {roc_auc}")
print(f"Balanced Accuracy: {balanced_accuracy}")
print(f"True Positive Rate: {tpr.mean()}")
print(f"False Positive Rate: {fpr.mean()}")
print(f"Thresholds: {thresholds}")

ROC AUC: 0.8518019574458421
Balanced Accuracy: 0.7080878929655267
True Positive Rate: 0.7147056114729294
False Positive Rate: 0.29852982554187607
Thresholds: [          inf 9.9104941e-01 9.7957587e-01 ... 3.9524937e-04 3.3891349e-04
 3.0215704e-04]


In [47]:
loss, accuracy,precision, recall, auc  = best_model.evaluate(X_h, y_h)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"Test AUC: {auc}")

295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - accuracy: 0.7885 - auc_3: 0.8515 - loss: 0.5233 - precision_3: 0.7625 - recall_3: 0.7614
Test Loss: 0.5166563987731934
Test Accuracy: 0.7908974885940552
Test Precision: 0.7401185631752014
Test Recall: 0.7652618288993835
Test AUC: 0.8518397808074951


In [38]:
df_htest_2 = pd.read_csv('25k_huggingface.csv')

In [40]:
# Identify rows where 'generated' == 'generated'
rows_to_drop = df_htest_2[df_htest_2['generated'] == 'generated'].index

# Drop these rows
dataset_h2 = df_htest_2.drop(rows_to_drop)

In [42]:
text_embeddings_h2 = model.encode(dataset_h2['text'].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 744/744 [03:44<00:00,  3.32it/s]


In [ ]:
X_h2 = text_embeddings_h2
y_h2 = df_h_distinct['generated']
y_h2.astype(int)  # Target column
X_h2 = np.array(X_h2, dtype=np.float32)  # Convert X to float32
y_h2 = np.array(y_h2, dtype=np.float32)